In [ ]:
# VERSION WITH AudioReader

# Import audio processing modules

%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import audioread
import librosa
import os
# import seaborn as sns

from audio_read import AudioReader

In [ ]:
def half_rectify(n):
    return np.fmax(n, np.zeros_like(n))

In [ ]:
def spectral_flux(spec):
    """
    Computes the spectral flux of a spectrogram
    :param spec: a 2-D array of floats
    :return: a numpy array
    """
    return np.expand_dims(np.sum(np.square(half_rectify(np.absolute(spec[:, 1:]) -
                                         np.absolute(spec[:, :-1]))), 0), axis=0)

In [ ]:
# Import ML modules

import sklearn.neighbors as nbrs
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

In [ ]:
# ML params - set dataset and algorithm

sample_dirs = ["../audio/samples/onsets/ALFRED/", "../audio/samples/onsets/DANBY/"]

# clf_KNN = nbrs.KNeighborsClassifier(n_neighbors=5)
# clf_SVM = SVC()
# clf_forest = RandomForestClassifier(verbose=1, warm_start=True)
clf_SGD = SGDClassifier(loss='log', penalty='l2', verbose=0, fit_intercept=False)

# clfs = [clf_KNN, clf_SVM, clf_forest]
clfs = [clf_SGD]
# clfs = [clf_forest]

In [ ]:
# Audio params

# filename = "../audio/SBI-1_20090915_234016.wav"
# annotation_path = "../annotations/SBI-1_20090915_HAND_LOW_IDaek_EDITED_with_HIGH.txt"
# model_path = "../features/SBI_coefs_41.npy"

# Load audio and compute spectrogram
sr = 24000.0
n_fft = 256 # =win_length
win_length = n_fft
hop_length = 128
hop_size = hop_length/sr # in seconds
win_size = win_length/sr # in seconds
spec_dt = hop_length/sr # in seconds
truncate = 64  # throw out freq bins below this index

# Define length of coefficient window
w_length = 1    # let's just use odd numbered values, ok?

In [ ]:
# Convert segment of spectrogram into ML features

def make_feature(spec, spec_dt, w_length):
    l = spec.shape[1]
    mid = l/2;
    start = mid - np.floor((w_length)/2)
    end = start + w_length

    # Choose which version of SF to use
#     sf = np.square(half_rectify(np.absolute(spec[:, start:end]) -
#                                      np.absolute(spec[:, start-1:end-1])))
    sf = (half_rectify(np.absolute(spec[:, start:end]) -
                                     np.absolute(spec[:, start-1:end-1])))
    return sf.reshape(1, sf.size)

In [ ]:
# y, _ = librosa.load("../audio/samples/onsets/ALFRED/true_123676.wav", sr=sr)
# spec = np.flipud(np.abs(librosa.core.stft(y, n_fft=n_fft, hop_length=hop_length)))

# plt.figure(figsize=(10,5))
# plt.imshow(spec)
# # plt.plot(sf)
# plt.imshow(spec[40:, start:end])
# # plt.imshow(spec[40:, start-1:end-1])

In [ ]:
# Train
import random

n_epochs = 50
for epoch in range(n_epochs):
    print "Epoch " + str(epoch)
    for sample_dir in sample_dirs:
            d = os.listdir(sample_dir)
#             random.shuffle(d)
            for f in d:
                if f[-4:] != '.wav':
                    continue
                y, _ = librosa.load(sample_dir + f, sr=sr)

                # Make feature
                spec = librosa.core.stft(y, n_fft=n_fft, hop_length=hop_length)
                feature = make_feature(spec[truncate:,:], spec_dt, w_length)

                # Make label
                if f[0] == 't':
                    label = np.asarray([1])
                elif f[0] == 'f':
                    label = np.asarray([0])
                else:
                    print "BAD FILENAMES!"
                    exit()
                
                # Fit feature
                for clf in clfs:
#                     clf.partial_fit(feature, label, classes=[0,1])
                    clf.fit(feature, label)
# return clfs
print "All done!"

In [ ]:
# Debuggy stuff

model = clfs[0]
print model
print w_length
coefs2 = model.coef_.reshape((n_fft/2 + 1 - truncate, w_length))
coefs = model.coef_
# np.save('model_59.npy', coefs)
print coefs.shape
plt.imshow((coefs2), interpolation='nearest', aspect='auto', cmap='hot')

np.argmax(coefs)

In [ ]:
def make_feature_set(spec, spec_dt, annotation_path, w_length):
    """
    Generates features and labels from spectrogram.
    :param spec: NxM numpy array
    :param spec_dt: float
    :param annotation_path: String
    :param w_length: int
    :return: features, labels
    """
    if annotation_path is not None:
        df = pd.read_csv(annotation_path, header=None, 
            names=['onsets', 'offsets', 'label'], delimiter='\t')
        onsets = np.asarray(df['onsets'])
        offsets = np.asarray(df['offsets'])

    num_features = spec.shape[1]-w_length+1

    # Generate features
    features = [[]] * num_features
    for i in xrange(num_features):
    	features[i] = np.ravel(spec[:, i:i+w_length])

    # Generate labels
    if annotation_path is not None:
        labels = np.zeros(num_features)
        for on in onsets:
            start = int(np.round((on-0.100)/spec_dt))
            finish = int(np.round((on+0.100)/spec_dt))
            labels[start:finish+1] = 1
        return np.asarray(features).T, labels

    return np.asarray(features).T

In [ ]:
######## Evaluate model
test_path = '../audio/NSDNS_20110902_192900.wav'
detection_curve_path = '../detection_functions/NSDNS_SF_71.npy'
# test_path = '../audio/ALFRED_20110924_183200.wav'
# detection_curve_path = '../detection_functions/ALFRED_SF_50_test.npy'

# Optional - load model
# coefs = np.load(model_path)
# c = coefs.reshape(645,1)

# Iterate through signal by large blocks (constrained by RAM)
duration = None          #seconds
num_hops_per_block = 30000    # Hops
block_len = (num_hops_per_block+w_length-1)*hop_length # Samples
block_size = 1.0*block_len/sr # Seconds

# Preallocate detection curve to save time
if duration is None:
    # If duration is None, we are looking at whole file. We use audioread to
    # check out the wav header and find the duration and sample rate
    with audioread.audio_open(test_path) as f:
        det_curve_len = int(np.floor(f.duration*f.samplerate/(hop_length*num_hops_per_block))*num_hops_per_block)
else:
    # Else we know the duration; we can calculate the detection curve's length directly
    det_curve_len = int(np.floor(duration*sr/(hop_length*num_hops_per_block))*num_hops_per_block)
detection_curve = np.zeros((1,det_curve_len))
print det_curve_len


block_i = 0
reader = AudioReader(test_path, sr=sr, channels=1)

print "Starting eval with params duration={}, block_len={}".format(duration, num_hops_per_block)

while not reader.done():
    # Read the next chunk of samples.
    print "Testing next block... i={} ".format(block_i) + str(time.clock())
    y, sr = reader.read(block_size)
    if len(y) < block_size*sr or (duration is not None and offset > duration):
        print "last one!"
        # Not quite right - throwing out last bit of data
        break
#         done = True
    D = librosa.core.stft(y, n_fft=n_fft, hop_length=hop_length)

    # Transform frames of spec into vector
    features = np.asarray(make_feature_set(D[truncate:,:], spec_dt, annotation_path=None, 
        w_length=w_length))

    # Apply almost all of SF algorithm
    actual_features = np.square(half_rectify(np.absolute(features[:, 1:]) -
                                     np.absolute(features[:, :-1])))

    # Weighted SF
    detection_curve[0][num_hops_per_block*block_i:num_hops_per_block*(block_i+1)] = np.dot(coefs, actual_features)    
    
    # Unweighted SF
#     detection_curve[0][num_hops_per_block*block_i:num_hops_per_block*(block_i+1)] = np.dot(np.ones_like(coefs), actual_features) 

    reader.sample_position -= w_length*hop_length
    block_i += 1

print spec_dt

In [ ]:
# print test_res.shape
# print sf_res.shape

# diff_res = test_res-sf_res
# print np.min(diff_res)
# print np.max(diff_res)
# plt.imshow(diff_res[:,:100])

In [ ]:
print detection_curve.shape
print detection_curve[:10]
# print detection_curve[0,:]
# detection_curve_path += 'off1'
detection_curve_path = '../detection_functions/NSDNS_SF_71_off3.npy'

print detection_curve_path
np.save(detection_curve_path, detection_curve[0,3:])

In [ ]:
# detection_curve = detection_curve[1:]
# detection_curve -= min(detection_curve)
# detection_curve /= max(detection_curve)
# plt.plot(detection_curve[0,29000:31000])
plt.plot(detection_curve[0,:])
# onsets = np.nonzero(actual_labels)
# plt.plot([onsets[0],onsets[0]], [-40,40], 'r')
plt.show()

In [ ]:
np.argmax(detection_curve)

In [ ]:
actual_labels[6280:6295]
np.nonzero(actual_labels)

In [ ]:
##### This is a separate program. Visualizes the average spectrogram for all 
#   audio containing/not containing flight calls

true_spec = None
false_spec = None
for sample_dir in sample_dirs:
    d = os.listdir(sample_dir)
    for f in d:
        if f[-4:] != '.wav':
            continue
        y, _ = librosa.load(sample_dir + f, sr=sr)

        # Make feature
        spec = librosa.core.stft(y, n_fft=n_fft, hop_length=hop_length)
#         feature = make_feature(spec[truncate:,:], spec_dt, w_length)

        # Make label
        if f[0] == 't':
            if true_spec is None:
                true_spec = spec
            else:
                true_spec += spec
        elif f[0] == 'f':
            if false_spec is None:
                false_spec = spec
            else:
                false_spec += spec
        else:
            print "BAD FILENAMES!"
            exit()

In [ ]:
# normalize
true_spec = np.abs(true_spec)
false_spec = np.abs(false_spec)
true_spec /= np.max(true_spec)
false_spec /= np.max(false_spec)

In [ ]:
plt.imshow(np.log(true_spec))

In [ ]:
plt.imshow(np.log(false_spec))

In [ ]:
diff_spec = ((np.log(true_spec)-np.log(false_spec)))[:,20:40]
plt.imshow(diff_spec/np.max(diff_spec))